In [2]:
!pip install torch transformers datasets scikit-learn pandas numpy tqdm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.4 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.


In [3]:
import numpy as np
import torch
from tqdm import tqdm
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, BertModel
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
from scipy.special import softmax

In [6]:
# Load the IMDB dataset
dataset = load_dataset("IMDB Dataset.csv")
df = pd.DataFrame(dataset['train'])

HFValidationError: Repo id must use alphanumeric chars or '-', '_', '.', '--' and '..' are forbidden, '-' and '.' cannot start or end the name, max length is 96: 'IMDB Dataset.csv'.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Create a balanced sample of 200 (100 positive and 100 negative)
df_positive = df[df['label'] == 1].sample(100, random_state=42)
df_negative = df[df['label'] == 0].sample(100, random_state=42)
balanced_df = pd.concat([df_positive, df_negative])

In [ ]:
# Extract the texts and labels
texts = balanced_df['text'].tolist()
labels = balanced_df['label'].tolist()

# Load the pre-trained BERT tokenizer and model for sequence classification

In [ ]:
# Load the pre-trained BERT tokenizer and model for sequence classification


MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
bert_model = AutoModelForSequenceClassification.from_pretrained(MODEL)

In [ ]:
def return_sentiment(txt):
    encoded_input = tokenizer(txt, return_tensors='pt', padding=True, truncation=True, max_length=510)
    output = bert_model(**encoded_input)
    score = output[0][0].detach().numpy()
    scores = softmax(score)

    # Get the predicted class (0 for negative, 1 for positive)
    if scores[2] > scores[0]:
      return 1

    return 0

In [ ]:
# Predict using BERT
predictions = []

for text in tqdm(texts):
  predictions.append(return_sentiment(text))


In [ ]:
# Create a confusion matrix for BERT model predictions
print("Confusion Matrix for BERT model:")
bert_conf_matrix = confusion_matrix(labels, predictions)
print(bert_conf_matrix)
print(classification_report(labels, predictions))

# Extract word embeddings from BERT and use Logistic Regression

In [ ]:
# Extract word embeddings from BERT and use Logistic Regression
lrtokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Tokenize the texts
inputs = lrtokenizer(texts, padding=True, truncation=True, return_tensors="pt")
bert_model = BertModel.from_pretrained("bert-base-uncased")

inputs = lrtokenizer(texts, padding=True, truncation=True, return_tensors="pt",max_length=510)
with torch.no_grad():
    embeddings = bert_model(**inputs).last_hidden_state[:, 0, :].numpy()  # Take the [CLS] token

In [ ]:
# Split the embeddings for training and testing the logistic regression
X_train, X_test, y_train, y_test = train_test_split(embeddings, labels, test_size=0.2, random_state=42)


In [ ]:

# Train a logistic regression model
lr_model = LogisticRegression()
lr_model.fit(X_train, y_train)

In [ ]:
# Predict using the logistic regression model
lr_predictions = lr_model.predict(X_test)

In [ ]:
# Create a confusion matrix for logistic regression predictions
print("Confusion Matrix for Logistic Regression model:")
lr_conf_matrix = confusion_matrix(y_test, lr_predictions)
print(lr_conf_matrix)
print(classification_report(y_test, lr_predictions))